# El Gamal Encryption Algorithm

El Gamal is an asymmetric key encryption algorithm based on the difficulty of the discrete logarithm problem. It was described by Taher Elgamal in 1985. It is commonly used in free software GNU Privacy Guard (GPG) and other cryptosystems.

## What is El Gamal?

El Gamal is a public-key cryptosystem, meaning it uses a pair of keys: a public key for encryption and a private key for decryption. It can be used for both encryption and digital signatures. Its security relies on the computational difficulty of finding discrete logarithms in a finite field, which is a hard problem for large prime numbers.

## Applications of El Gamal

*   **Secure Communication:** Encrypting messages to ensure confidentiality between parties.
*   **Digital Signatures:** Verifying the authenticity and integrity of messages.
*   **Key Exchange:** Establishing shared secret keys over insecure channels.
*   **GNU Privacy Guard (GPG):** A widely used open-source encryption suite that supports El Gamal.

<center> <img src="https://media.geeksforgeeks.org/wp-content/uploads/20240625160232/ElGamal-Encryption-Algorithm-(1).png"> </center>

## Mathematical Concepts - El Gamal

El Gamal is based on modular arithmetic and the properties of cyclic groups. Key concepts include:

1.  **Large Prime `p`:** A large prime number that defines the finite field.
2.  **Generator `g`:** An integer that is a primitive root modulo `p`. This means that powers of `g` modulo `p` generate all integers from 1 to `p-1`.
3.  **Private Key `x`:** A randomly chosen integer such that `1 < x < p-1`.
4.  **Public Key `y`:** Calculated as `g^x mod p`.

### Key Generation

*   **Choose a large prime `p` and a generator `g`** of the multiplicative group of integers modulo `p`.
*   **Choose a random integer `x`** (private key) where `1 < x < p-1`.
*   **Compute `y = g^x mod p`** (public key).
*   The public key is `(p, g, y)`. The private key is `x`.

### Encryption

To encrypt a message `M` for a recipient with public key `(p, g, y)`:

*   **Choose a random integer `k`** (ephemeral key) where `1 < k < p-1`.
*   **Compute `c1 = g^k mod p`**.
*   **Compute `s = y^k mod p`** (shared secret).
*   **Compute `c2 = (M * s) mod p`**.
*   The ciphertext is `(c1, c2)`.

### Decryption

To decrypt a ciphertext `(c1, c2)` using the private key `x`:

*   **Compute `s = c1^x mod p`** (shared secret).
*   **Compute `s_inv = s^(p-2) mod p`** (multiplicative inverse of `s` modulo `p` using Fermat's Little Theorem, as `p` is prime).
*   **Compute `M = (c2 * s_inv) mod p`**.
*   The decrypted message is `M`.

In [1]:
#Import Library
import random

In [2]:
#Create the Prime Functions
def is_prime(n):
    """Checks if a number is prime."""
    if n < 2: return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0: return False
    return True

def generate_prime(bits=16):
    """Generates a prime number of a given bit length."""
    while True:
        p = random.getrandbits(bits)
        if is_prime(p): return p

def mod_pow(base, exp, mod):
    """Calculates (base^exp) % mod efficiently."""
    res = 1
    base %= mod
    while exp > 0:
        if exp % 2 == 1: res = (res * base) % mod
        base = (base * base) % mod
        exp //= 2
    return res

def find_generator(p):
    """Finds a generator (primitive root) for a prime p."""
    if p == 2: return 1
    # phi = p - 1
    phi = p - 1
    factors = set()
    temp = phi
    d = 2
    while d * d <= temp:
        if temp % d == 0:
            factors.add(d)
            while temp % d == 0:
                temp //= d
        d += 1
    if temp > 1: factors.add(temp)

    for g in range(2, p):
        is_generator = True
        for factor in factors:
            if mod_pow(g, phi // factor, p) == 1:
                is_generator = False
                break
        if is_generator: return g
    return None # Should not happen for a prime p

In [3]:
#Create the Keys Manipulation Function
def generate_elgamal_keys(bits=16):
    """Generates El Gamal public and private keys."""
    p = generate_prime(bits)
    g = find_generator(p)
    x = random.randint(2, p - 2) # Private key
    y = mod_pow(g, x, p)      # Public key part

    public_key = (p, g, y)
    private_key = x
    return public_key, private_key

def encrypt(message, public_key):
    """Encrypts a message using El Gamal public key."""
    p, g, y = public_key
    k = random.randint(2, p - 2) # Ephemeral key
    c1 = mod_pow(g, k, p)
    s = mod_pow(y, k, p) # Shared secret
    c2 = (message * s) % p
    return (c1, c2)

def decrypt(ciphertext, private_key, public_key):
    """Decrypts a ciphertext using El Gamal private key."""
    c1, c2 = ciphertext
    p, g, y = public_key # Need p for modular inverse
    x = private_key

    s = mod_pow(c1, x, p) # Shared secret
    # Calculate s_inv = s^(p-2) mod p using Fermat's Little Theorem
    s_inv = mod_pow(s, p - 2, p)
    message = (c2 * s_inv) % p
    return message

In [4]:
#El Gamal Example of Application
public_key, private_key = generate_elgamal_keys(bits=8)
p, g, y = public_key
print(f"Public Key (p, g, y): ({p}, {g}, {y})")
print(f"Private Key (x): {private_key}\n")

Public Key (p, g, y): (19, 2, 4)
Private Key (x): 2



In [5]:
#Text to Encrypt (must be an integer less than p)
original_message = 123
print(f"Original Message: {original_message}\n")

Original Message: 123



In [6]:
#Message Validation
if original_message >= p:
    print(f"Error: Message {original_message} must be less than prime p ({p}).\n")
    original_message = random.randint(1, p-1)
    print(f"Adjusted message to: {original_message}\n")

Error: Message 123 must be less than prime p (19).

Adjusted message to: 13



In [7]:
#Encryption
c1, c2 = encrypt(original_message, public_key)
print(f"Ciphertext (c1, c2): ({c1}, {c2})\n")

Ciphertext (c1, c2): (11, 15)



In [8]:
#Decryption
decrypted_message = decrypt((c1, c2), private_key, public_key)
print(f"Decrypted Message: {decrypted_message}\n")

Decrypted Message: 13



In [9]:
#Validation
if original_message == decrypted_message:
    print("Encryption and Decryption Successful!")
else:
    print("Error: Decryption failed.")

Encryption and Decryption Successful!
